In [9]:
# using Google Colab environment
from google.colab import drive
drive.mount('/content/drive')

import glob
import pandas as pd

# Calling Google Colab to access the folder in the Google Drive
folder_path = '/content/drive/MyDrive/bike-rental-starter-kit/data/JC-201*.csv'
citibike_files = glob.glob(folder_path)
dfs = [pd.read_csv(file) for file in citibike_files]

# concatenating 12 citibike data files into one
citibike_raw = pd.concat(dfs, ignore_index=True)

# reading in the weather data from the same folder
weather_raw = pd.read_csv('/content/drive/MyDrive/bike-rental-starter-kit/data/newark_airport_2016.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
citibike_raw.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0


In [11]:
weather_raw.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


In [12]:
import numpy as np
import pandas as pd

def _haversine_km(lat1, lon1, lat2, lon2):
    """Haversine distance in kilometers."""
    R = 6371.0088
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))


# ==== 1) Citibike cleaning ====
def clean_citibike(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # --- column renaming ---
    rename_map = {
        'Trip Duration': 'trip_duration_seconds',
        'Start Time': 'start_time',
        'Stop Time': 'stop_time',
        'Start Station ID': 'start_station_id',
        'Start Station Name': 'start_station_name',
        'Start Station Latitude': 'start_latitude',
        'Start Station Longitude': 'start_longitude',
        'End Station ID': 'end_station_id',
        'End Station Name': 'end_station_name',
        'End Station Latitude': 'end_latitude',
        'End Station Longitude': 'end_longitude',
        'Bike ID': 'bike_id',
        'User Type': 'user_type',
        'Birth Year': 'birth_year',
        'Gender': 'gender',
        'Trip Duration (minutes)': 'trip_duration_minutes',
        'Latitude Change': 'latitude_change',
        'Longitude Change': 'longitude_change',
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # --- dtypes & derived duration ---
    # times
    if 'start_time' in df: df['start_time'] = pd.to_datetime(df['start_time'])
    if 'stop_time' in df:  df['stop_time']  = pd.to_datetime(df['stop_time'])

    # numeric fields (coerce errors to NaN)
    num_cols = [
        'trip_duration_seconds','start_station_id','end_station_id','bike_id',
        'start_latitude','start_longitude','end_latitude','end_longitude',
        'birth_year','gender'
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    # minutes
    if 'trip_duration_minutes' not in df.columns and 'trip_duration_seconds' in df.columns:
        df['trip_duration_minutes'] = df['trip_duration_seconds'] / 60.0

    # --- gender mapping (1->M, 2->F, 0/NaN->NaN) ---
    if 'gender' in df.columns:
        df['gender'] = df['gender'].replace({1: 'M', 2: 'F', 0: pd.NA})

    # --- reasonable trip duration filter (1 min .. 24 hours) ---
    if 'trip_duration_minutes' in df.columns:
        df = df[(df['trip_duration_minutes'] >= 1) & (df['trip_duration_minutes'] <= 24*60)]

    # --- tighten lat/long bounds to NYC-area  ---
    LAT_MIN, LAT_MAX = 40.3, 41.2
    LON_MIN, LON_MAX = -74.5, -73.3
    for col in ['start_latitude','end_latitude']:
        if col in df.columns:
            df = df[df[col].between(LAT_MIN, LAT_MAX)]
    for col in ['start_longitude','end_longitude']:
        if col in df.columns:
            df = df[df[col].between(LON_MIN, LON_MAX)]

    # --- station name canonicalization (trim, de-dup spaces, Title Case) ---
    for col in ['start_station_name', 'end_station_name']:
        if col in df.columns:
            df[col] = (
                df[col].astype('string')
                      .str.strip()
                      .str.replace(r'\s+', ' ', regex=True)
                      .str.title()
            )

    # --- age (10..100; else NaN) ---
    if {'birth_year','start_time'}.issubset(df.columns):
        current_year = pd.Timestamp(df['start_time'].max()).year
        df['age'] = current_year - df['birth_year']
        df.loc[(df['age'] < 10) | (df['age'] > 100), 'age'] = np.nan

    # --- distance (km) & speed (km/h) & sanity filters you used ---
    if {'start_latitude','start_longitude','end_latitude','end_longitude'}.issubset(df.columns):
        df['distance_km'] = _haversine_km(
            df['start_latitude'], df['start_longitude'],
            df['end_latitude'],   df['end_longitude']
        )

    if {'distance_km','trip_duration_minutes'}.issubset(df.columns):
        duration_hours = (df['trip_duration_minutes'] / 60.0).replace(0, np.nan)
        df['speed_kmh'] = df['distance_km'] / duration_hours

        # Filter out impossible speeds (> 60 km/h) and long-duration zero-distance trips
        df = df[
            (df['speed_kmh'] <= 60) &
            ~((df['distance_km'] == 0) & (df['trip_duration_minutes'] > 10))
        ]

    # --- extra time fields useful for analysis ---
    if 'start_time' in df:
        df['start_hour']    = df['start_time'].dt.hour
        df['start_weekday'] = df['start_time'].dt.day_name()
        df['is_weekend']    = df['start_time'].dt.weekday >= 5
        df['rush_hour']     = df['start_hour'].between(7,9) | df['start_hour'].between(16,19)
        df['start_date']    = df['start_time'].dt.date

    # lat/lon deltas
    if {'start_latitude','end_latitude'}.issubset(df.columns) and 'latitude_change' not in df:
        df['latitude_change']  = df['end_latitude']  - df['start_latitude']
    if {'start_longitude','end_longitude'}.issubset(df.columns) and 'longitude_change' not in df:
        df['longitude_change'] = df['end_longitude'] - df['start_longitude']

    return df

In [13]:
# ==== 2) Weather cleaning ====
def clean_weather(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # drop columns that are completely null (PGTM, TSUN)
    all_null_cols = df.columns[df.isna().all()]
    df = df.drop(columns=all_null_cols)

    # parse date
    if 'DATE' in df.columns:
        df['DATE'] = pd.to_datetime(df['DATE'])

    # rename columns to descriptive names better for analysis
    rename_map = {
        'STATION': 'station',
        'NAME': 'name',
        'DATE': 'date',
        'AWND': 'average_daily_wind_speed_mph',
        'PRCP': 'precipitation_in',
        'SNOW': 'snowfall_in',
        'SNWD': 'snow_depth_in',
        'TAVG': 'average_temp_f',
        'TMAX': 'max_temp_f',
        'TMIN': 'min_temp_f',
        'WDF2': 'wind_direction_fastest_2_min_degrees',
        'WSF2': 'wind_speed_fastest_2_min_mph',
        'WSF5': 'wind_speed_fastest_5_sec_mph',
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    return df

In [14]:
# --- run custom functions to clean datasets ---
citibike = clean_citibike(citibike_raw)
weather  = clean_weather(weather_raw)

# quick sanity check
display(citibike.head())
display(weather.head())

,trip_duration_seconds,start_time,stop_time,start_station_id,start_station_name,start_latitude,start_longitude,end_station_id,end_station_name,end_latitude,...,age,distance_km,speed_kmh,start_hour,start_weekday,is_weekend,rush_hour,start_date,latitude_change,longitude_change
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St Path,40.719586,-74.043117,3209,Brunswick St,40.724176,...,52.0,0.814973,8.104701,0,Friday,False,False,2016-01-01,0.004590,-0.007539
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St Path,40.719586,-74.043117,3213,Van Vorst Park,40.718489,...,54.0,0.407154,7.328768,0,Friday,False,False,2016-01-01,-0.001097,-0.004609
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St Path,40.719586,-74.043117,3213,Van Vorst Park,40.718489,...,54.0,0.407154,7.256206,0,Friday,False,False,2016-01-01,-0.001097,-0.004609
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,...,32.0,0.660501,9.587912,0,Friday,False,False,2016-01-01,0.003419,0.006409
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,...,NaN,1.668084,6.650168,1,Friday,False,False,2016-01-01,0.011935,0.011995


,station,name,date,average_daily_wind_speed_mph,precipitation_in,snowfall_in,snow_depth_in,average_temp_f,max_temp_f,min_temp_f,wind_direction_fastest_2_min_degrees,WDF5,wind_speed_fastest_2_min_mph,wind_speed_fastest_5_sec_mph
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,0.0,0.0,0.0,41,43,34,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,0.0,0.0,0.0,36,42,30,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,0.0,0.0,0.0,37,47,28,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,0.0,0.0,0.0,32,35,14,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,0.0,0.0,0.0,19,31,10,360,350.0,25.1,31.1


In [17]:
# --- saving cleaned datasets to local environment ---
citibike.to_csv('citibike_cleaned.csv', index=False)
weather.to_csv('weather_cleaned.csv', index=False)

# downloading files to local computer
from google.colab import files
files.download('citibike_cleaned.csv')
files.download('weather_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>